# Theano / computing gradient numerically.. (not autodiff)

To evaluate how well a particular set of parameter values $\theta$ fits the behavioral data, we compute the probability of oberving the data given the model.

For each trial $i$, we will compute the likelihood of seeing the data under the model assuming that trials are independent. 

$P(D|\theta) = \prod_{i}P(d_i|t_{i,R},t_{i,L},\theta)$

$t_{i,R},t_{i,L}$ : the right and left click times on trial $i$

$d_i$ : the subject's decision on trial $i$

The best-fit parameter values are the parameters $\theta$ that maximize the likelihood (Maximum likelihood values)

To help maximize the likelihood(or log likelihood), we will compute the derivative $\partial P(d_i|t_{i,R},t_{i,L},\theta) / \partial\theta$ for each of the parameters in the set $\theta$.

After we get these gradients of 9 model parameters, we will apply them for optimization.

In [1]:
# data import
import scipy.io
ratdata = scipy.io.loadmat('chrono_B069_rawdata.mat')

In [2]:
# trial index starts from 0 
# *** rightbups, leftbups, maxT, rat_choice

def trialdata(ratdata,trial):
    if ratdata['rawdata']['pokedR'][0][trial] > 0 :
        rat_choice = 1 # "R"
    else : 
        rat_choice = -1 # "L"
        
    RClickTimes = np.require(ratdata['rawdata']['rightbups'][0][trial][0],requirements='A')
    LClickTimes = np.require(ratdata['rawdata']['leftbups'][0][trial][0],requirements='A')
    T1 = ratdata['rawdata']['T'][0][trial][0][0]
    
    if np.shape(LClickTimes)[0] == 1:
        LClickTimes = [0, LClickTimes]
    if np.shape(RClickTimes)[0] == 1:
        RClickTimes = [0, RClickTimes]
    return  RClickTimes, LClickTimes, T1, rat_choice

In [3]:
# import packages..
# pip install theano

import numpy as np
import theano
import theano.tensor as T
from theano.compile.nanguardmode import NanGuardMode
from theano.compile.debugmode import DebugMode

### Configuration

In [4]:
theano.config.mode = 'FAST_RUN'

In [5]:
# trial index starts from 0 
# ** data load sample
RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, 0) 
print LeftClickTimes
print maxT
print rat_choice

[ 0.       0.19235  0.34361]
0.350959
-1



### Variables (Global variable, Theano shared variables = Parameters)


a : decision variable, memory accumulator

$$ da =
  \begin{cases}
    0       & \quad \text{if, } |a| \geq B \\
    \sigma_adW + (\delta_{t,t_R} \cdot \eta C(t) - \delta_{t,t_L} \cdot \eta C(t))dt + \lambda adt  & \quad \text{otherwise, }\\
  \end{cases}
$$



The impact of each click (C) is affected by sensory adaptation that depends on clicks from both right and left sides:

$$ 
\frac{\mathrm d C}{\mathrm d t} = \frac{1-C}{\tau_\phi} + (1-\phi)C(\delta_{t,t_R}+\delta_{t,t_L}) 
$$


sigma2_a ($\sigma_a^2$) : a diffusion constant, parameterizing noise in a.

sigma2_s ($\sigma_s^2$) : parameterizing noise when adding evidence from a right or left pulse. (incoming sensory evidence)

sigma2_i ($\sigma_i^2$) : initial condition for the dynamical equation at $t=0$

lam ($\lambda$) : consistent drift in the memory a ($\lambda<0$ : leaky or forgetful case, $\lambda>0$ : unstable or impulsive case)

B : decision bound

bias : bias parameter determines the position of the threshold in a (which a Rightward decision is made)

phi ($\phi$) : parameterize sensory adaptation (by defining the dynamics of C ($\phi>1$ : Facilitation, $\phi<1$ : Depression, $\phi=1$ : absense of sensory adaptation)

tau_phi ($\tau_\phi$) :

lapse : The lapse rate parameterizes the probability of making a random response.



In [6]:
# Global variables 
epsilon = 10.0**(-10) 
dx = 0.25
dt = 0.02
# new_global var
ndelta = 10.0**(-4)
total_rate = 40

# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.1;

# Theano shared_variables 
sigma2_a = theano.shared(sigma_a, name="sigma_a")
sigma2_s = theano.shared(sigma_s, name="sigma_s")
sigma2_i = theano.shared(sigma_i, name="sigma_i")
lam = theano.shared(lam, name="lambda")
B = theano.shared(B, name="B")
bias = theano.shared(bias, name="bias")
phi = theano.shared(phi, name="phi")
tau_phi = theano.shared(tau_phi, name="tau_phi")
lapse = theano.shared(lapse, name="lapse")

params = [sigma2_a, sigma2_s, sigma2_i, lam, B, bias, phi, tau_phi, lapse]

## why binning?

Because of the bounds B, we can't use analytical solutions for P(a,t)

Instead, we will solve the Fokker-Planck equations numerically.

1. Discretize both time t and space a
2. Start with the distribution at time t=0 : $P(a, t=0) = N(0,\sigma_i)$
3. Compute the probability distribution for the next time step given the probability distribution for the previous time step. (until t = maxT)

*time step k, $a_k, t_k$ : at time step k

*a -> a set of M contiguous bin (space), vector $\xi$ (center of bins), $\xi_i, \xi_j$

### Forward Markov transition matrix F

$F_{ij} = P(a_k = \xi_i | a_{k-1} = \xi_j)$

$f_k$ represents the spatially discretized version of the distribution $P(a, t_k)$

$ f_k = Ff_{k-1}$

## bin_centers = make_bins(B, dx, binN)

In [7]:
#### make_bins (Theano version)
def make_bins(B, dx, binN):
    bins = (T.arange(binN)+1)*B
    bins = dx*bins/B

    tmp = T.scalar()
    
    bins = T.switch(T.eq(bins[-1],B),
                    T.set_subtensor(bins[-1], B+dx),
                    T.set_subtensor(bins[-1], 2*B - bins[-2]))
    
    bins = T.concatenate((-bins[::-1], T.zeros(1), bins))
    return bins

In [8]:
binN = T.cast(T.ceil(B/dx),'int32')
bin_centers = make_bins(B, dx, binN)
test = T.zeros_like(bin_centers)
test2 = T.zeros((bin_centers.shape))

test_func = theano.function(
    inputs=[],
    outputs=[bin_centers, binN,test,test2]
)

test_func()

[array([-4.2 , -4.  , -3.75, -3.5 , -3.25, -3.  , -2.75, -2.5 , -2.25,
        -2.  , -1.75, -1.5 , -1.25, -1.  , -0.75, -0.5 , -0.25,  0.  ,
         0.25,  0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ,  2.25,
         2.5 ,  2.75,  3.  ,  3.25,  3.5 ,  3.75,  4.  ,  4.2 ]),
 array(17, dtype=int32),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])]

## F = Fmatrix([sigma, lambda, c], bin_centers)

In [9]:
def inner_loop(base_s, s, p, hp, lp, F, dFds, dFdsig2, dFdB, bin_centers,j,sigma2):
    dd = bin_centers[hp] - bin_centers[lp]
    
    # ------ F
    FF = T.inc_subtensor(F[hp,j], p*(s-bin_centers[lp])/dd)
    FF = T.inc_subtensor(FF[lp,j], p*(bin_centers[hp]-s)/dd)

    F_rest = T.switch(T.eq(dd,0),
                      T.inc_subtensor(F[lp,j], p),
                      FF)

    F = T.switch(T.le(s,bin_centers[0]),
                      T.inc_subtensor(F[0,j], p),
                      F)
    F = T.switch(T.ge(s,bin_centers[-1]),
                      T.inc_subtensor(F[-1,j], p),
                      F)
    F = T.switch(T.gt(s,bin_centers[0]) & T.lt(s,bin_centers[-1]),
                      F_rest,
                      F)

    # ------ dFds
    
    # hp == lp
    dFds1 = T.inc_subtensor(dFds[lp,j], -p/(bin_centers[lp+1]-bin_centers[lp]))
    dFds1 = T.inc_subtensor(dFds1[lp+1,j], p/(bin_centers[lp+1]-bin_centers[lp]))

    # hp != lp
    dFds2 = T.inc_subtensor(dFds[hp,j], p/dd)
    dFds2 = T.inc_subtensor(dFds2[lp,j], -p/dd)     
    
    dFds_rest = T.switch(T.eq(dd,0),
                      dFds1,
                      dFds2)
    
    dFds = T.switch(T.gt(s,bin_centers[0]) & T.lt(s,bin_centers[-1]),
                  dFds_rest,
                  dFds)
    
    # ------ dFdsig2
    # hp == lp
    dFdsig2_1 = T.inc_subtensor(dFdsig2[lp,j], -(p/(bin_centers[lp+1]-bin_centers[lp]))*base_s/(2*sigma2))
    dFdsig2_1 = T.inc_subtensor(dFdsig2_1[lp+1,j], (p/(bin_centers[lp+1]-bin_centers[lp]))*base_s/(2*sigma2))


    # hp != lp
    dFdsig2_2 = T.inc_subtensor(dFdsig2[hp,j], (p/dd)*base_s/(2*sigma2))
    dFdsig2_2 = T.inc_subtensor(dFdsig2_2[lp,j], -(p/dd)*base_s/(2*sigma2))
    
    dFdsig2_rest = T.switch(T.eq(dd,0),
                      dFdsig2_1,
                      dFdsig2_2)
    
    dFdsig2 = T.switch(T.gt(s,bin_centers[0]) & T.lt(s,bin_centers[-1]),
                  dFdsig2_rest,
                  dFdsig2)    

    # ------ dFdB
    # hp == lp
    # dFdB
    
    # hp != lp
    #  lp == 0
    dFdB2 = T.inc_subtensor(dFdB[lp,j], -2*p*(bin_centers[hp]-s)/dd**2)
    dFdB2 = T.inc_subtensor(dFdB2[hp,j], -2*p*(s-bin_centers[lp]-dd)/dd**2)

    dFdB_rest = T.switch(T.eq(lp,0),
                      dFdB2,
                      dFdB)  
    
    #  hp == bin_centers.shape[0]-1
    dFdB3 = T.inc_subtensor(dFdB_rest[lp,j], 2*p*(dd-(bin_centers[hp]-s))/dd**2)
    dFdB3 = T.inc_subtensor(dFdB3[hp,j], -2*p*(s-bin_centers[lp])/dd**2)
    
    dFdB_rest = T.switch(T.eq(hp,bin_centers.shape[0]-1),
                      dFdB3,
                      dFdB_rest)  
    
    dFdB = T.switch(T.eq(dd,0),
                      dFdB,
                      dFdB_rest)    

    ## To detect NaN !!!
    F = T.set_subtensor(F[(T.isnan(F)).nonzero()],0)
    
    return F, dFds, dFdsig2, dFdB

In [10]:
def outer_loop(mu,j, F, dFds, dFdsig2, dFdB, base_sbins, ps, bin_centers,sigma2):
    sbins = base_sbins + mu

    n = bin_centers.shape[0]

    hps = T.cast(T.ceil( (sbins-bin_centers[1])/dx) +1,'int32')
    lps = T.cast(T.floor((sbins-bin_centers[1])/dx) +1,'int32')
    
    hps = T.set_subtensor(hps[(hps < 0).nonzero()],0)
    lps = T.set_subtensor(lps[(lps < 0).nonzero()],0)

    hps = T.set_subtensor(hps[(hps > n-2).nonzero()],n-2)
    lps = T.set_subtensor(lps[(lps > n-2).nonzero()],n-2)
    
    hps = T.set_subtensor(hps[(bin_centers[-1]<sbins).nonzero()],n-2)
    lps = T.set_subtensor(lps[(bin_centers[-1]<sbins).nonzero()],n-2)

    hps = T.set_subtensor(hps[(sbins<bin_centers[0]).nonzero()],0)
    lps = T.set_subtensor(lps[(sbins<bin_centers[0]).nonzero()],0)


    hps = T.set_subtensor(hps[T.and_(bin_centers[0]<sbins, sbins<bin_centers[1]).nonzero()],1)
    lps = T.set_subtensor(lps[T.and_(bin_centers[0]<sbins, sbins<bin_centers[1]).nonzero()],0)
    
    hps = T.set_subtensor(hps[T.and_(bin_centers[-2]<sbins, sbins<bin_centers[-1]).nonzero()],n-1)
    lps = T.set_subtensor(lps[T.and_(bin_centers[-2]<sbins, sbins<bin_centers[-1]).nonzero()],n-2)


    hps = T.set_subtensor(hps[(T.isnan(hps)).nonzero()],0)
    lps = T.set_subtensor(lps[(T.isnan(lps)).nonzero()],0)    
    
    # sequences: sbins, ps, hps, lps // 
    # prior results: F //
    # non-sequences: bin_centers, index j // 
    [F, dFds, dFdsig2, dFdB],_ = theano.scan(fn=inner_loop,
                           outputs_info = [F, dFds, dFdsig2, dFdB],
                           sequences = [base_sbins, sbins, ps, hps, lps],
                           non_sequences = [bin_centers,j,sigma2],
                           name = 'Fmatrix_outer_loop_scan' 
                           )
    F = F[-1];
    dFds = dFds[-1]; dFdsig2 = dFdsig2[-1]; dFdB = dFdB[-1]; 

    F = T.set_subtensor(F[(T.isnan(F)).nonzero()],0)
    
    return F, dFds, dFdsig2, dFdB

In [11]:
#### Fmatrix (Theano version)
def Fmatrix_i(params, bin_centers):
    global dt,dx
    sigma2 = params[0]
    lam = params[1]
    c = params[2]
    
    
#     F = T.zeros((bin_centers.shape[0],bin_centers.shape[0]),'float32')
    
    # for Fi (when lambda = 0)
    mus = bin_centers*T.exp(lam*dt)

#     mus = T.exp(lam*dt)*(bin_centers + c/lam) - c/lam
        
    tmp = T.ceil(10*T.sqrt(sigma2)/dx)
    n_sbins = T.switch(T.ge(tmp,70),tmp,70) # ndeltas
    
    n_sbins = T.switch(T.eq(sigma2,0),1,n_sbins)
    
    swidth = 5*T.sqrt(sigma2)
    sbins = (T.arange(n_sbins)+1)/n_sbins*swidth#np.linspace(-swidth,swidth,n_sbins*2+1)
    sbins = T.concatenate((-sbins[::-1], T.zeros(1), sbins))

    sbins = T.switch(T.eq(sigma2,0),T.zeros(1),sbins)
    
    ps = T.exp(-sbins**2/(2*sigma2)) 
    ps = ps/T.sum(ps)
    
    ps = T.set_subtensor(ps[(T.isnan(ps)).nonzero()],0)
            
    base_sbins = sbins
        
    # sequences: mus[1:], array(1:binsize) // mus, indices of the outer loop
    # prior results: zeros(size(F)) // initialize F as zeros
    # non-sequences: base_bins, ps, bin_centers // 
    [res, dFds, dFdsig2, dFdB],_ = theano.scan(fn = outer_loop,
                      outputs_info = [T.zeros((bin_centers.shape[0],bin_centers.shape[0])), # F
                                      T.zeros((bin_centers.shape[0],bin_centers.shape[0])), # dFds
                                      T.zeros((bin_centers.shape[0],bin_centers.shape[0])), # dFdsig2
                                      T.zeros((bin_centers.shape[0],bin_centers.shape[0]))], # dFdB
                                      sequences = [mus[1:-1], T.arange(bin_centers.shape[0]-2)+1],
                      non_sequences = [base_sbins, ps, bin_centers,sigma2],
                      name = 'Fmatrix_i_scan'
                     )
    F = res[-1]
    dFds = dFds[-1]; dFdsig2 = dFdsig2[-1]; dFdB = dFdB[-1];
 
    # lambda == 0
    # ------ dFdl
    dFdl = T.zeros((bin_centers.shape[0],bin_centers.shape[0]))
    
    # ------ dFdh
    dFdh = dFds*dt
    
    F = T.set_subtensor(F[:,0], 0)
    F = T.set_subtensor(F[:,-1], 0)
    F = T.set_subtensor(F[0,0], 1)
    F = T.set_subtensor(F[-1,-1], 1)
    
    return F, dFdsig2, dFdB, dFdl, dFdh

In [18]:
#### Fmatrix (Theano version)
def Fmatrix(params, bin_centers):
    global dt,dx
    sigma2 = params[0]
    lam = params[1]
    c = params[2]
    
    sigma2_sbin = sigma2
    
#     F = T.zeros((bin_centers.shape[0],bin_centers.shape[0]),'float32')
    
#     mus = T.switch(T.eq(lam,0),bin_centers*T.exp(lam*dt),T.exp(lam*dt)*(bin_centers + c/lam) - c/lam)
    mus = T.exp(lam*dt)*(bin_centers + c/lam) - c/lam
        
    tmp = T.ceil(10*T.sqrt(sigma2_sbin)/dx)
    n_sbins = T.switch(T.ge(tmp,70),tmp,70)
    
    # slices of a gaussian with sigma2 variance
    sbin_ends = 5*T.sqrt(sigma2_sbin)
    sbins = (T.arange(n_sbins)+1)/n_sbins*sbin_ends
    sbins = T.concatenate((-sbins[::-1], T.zeros(1), sbins)) # np.linspace(-sbin_ends,sbin_ends,n_sbins*2+1)

    ps = T.exp(-sbins**2/(2*sigma2)) 
    ps = ps/T.sum(ps)
            
    base_sbins = sbins
              
    # sequences: mus[1:], array(1:binsize) // mus, indices of the outer loop
    # prior results: zeros(size(F)) // initialize F as zeros
    # non-sequences: base_bins, ps, bin_centers // 
    [res, dFds, dFdsig2, dFdB],_ = theano.scan(fn = outer_loop,
                      outputs_info = [T.zeros((bin_centers.shape[0],bin_centers.shape[0])), # F
                                      T.zeros((bin_centers.shape[0],bin_centers.shape[0])), # dFds
                                      T.zeros((bin_centers.shape[0],bin_centers.shape[0])), # dFdsig2
                                      T.zeros((bin_centers.shape[0],bin_centers.shape[0]))], # dFdB
                                      sequences = [mus[1:-1], T.arange(bin_centers.shape[0]-2)+1],
                      non_sequences = [base_sbins, ps, bin_centers,sigma2],
                      name = 'Fmatrix_scan'
                     )
    F = res[-1]
    dFds = dFds[-1]; dFdsig2 = dFdsig2[-1]; dFdB = dFdB[-1];

    # lambda > 0
    # ------ dFdl
    # gamma = T.exp(lam*dt), phi = c/lam
    dFdl = dFds*(bin_centers + c/lam)*dt*T.exp(lam*dt) - dFds*c/(lam**2)*(T.exp(lam*dt)-1) 
    
    # ------ dFdh
    dFdh = dFds*(T.exp(lam*dt)-1)/lam
    
    F = T.set_subtensor(F[:,0], 0)
    F = T.set_subtensor(F[:,-1], 0)
    F = T.set_subtensor(F[0,0], 1)
    F = T.set_subtensor(F[-1,-1], 1)
    
    return F, dFdsig2, dFdB, dFdl, dFdh

In [20]:
Fi= Fmatrix_i([sigma2_i, 0, 0.0], bin_centers)
 

test_func = theano.function(
    inputs=[],
    outputs=Fi
)

test_func()

[array([[ 1.        ,  0.41181999,  0.21890701, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.19714171,  0.17213129, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.18722765,  0.21792338, ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,  0.21792338,
          0.18722765,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.17213129,
          0.19714171,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.21890701,
          0.41181999,  1.        ]]),
 array([[ 0.        ,  0.22658477,  0.6279845 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , -0.46408419, -0.39048509, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , -0.34179845, -0.47499883, ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,

In [ ]:
F, dFds, dFdsig2, dFdB, dFdl, dFdh = Fmatrix([1.0000 ,0.1, 0.2], bin_centers)
 

test_func = theano.function(
    inputs=[],
    outputs=F
)

test_func()


## Backwards

In [14]:
def Bmatrix(F,Fdistribm1,Fdistrib, dFdsig2, dFdB, dFdl, dFdh, net_input, Pd):
    Bmat = F*Fdistribm1
    Bmat = T.transpose(Bmat)/Fdistrib
    Bmat = T.set_subtensor(Bmat[(T.isnan(Bmat)).nonzero()],0)
    
    # Joint Posterior probability 
    JmF = T.transpose(T.outer(Fdistribm1,Pd)/Fdistrib)
    
    Pd = T.dot(Bmat,Pd)
    
    JmF = T.set_subtensor(JmF[T.or_(T.isnan(JmF),T.isinf(JmF)).nonzero()],0)
    JmF = T.set_subtensor(JmF[T.eq(F,0).nonzero()],0)

    Deltak_l = T.sum(T.sum(JmF*dFdl))
    Deltak_h = T.switch(T.eq(net_input,0),0,T.sum(T.sum(JmF*dFdh)))
    Deltak_sig2 = T.sum(T.sum(JmF*dFdsig2))
    Deltak_B = T.sum(T.sum(JmF*dFdB))
 
    return JmF, Pd, Deltak_l, Deltak_h, Deltak_sig2, Deltak_B

def backwards(Pf, Pd, Fmats, dFdsig2, dFdB, dFdl, dFdh, net_input):
    # Pf 35x18
    # Fs 17x35x35
    
    # normalize to use as initial distribution of the backwards, posterior run
    Pd = Pd/T.sum(Pd)
    
    input_Fs = Fmats # for reversed sequence k-1 -> 1
    input_f_m1 = Pf[:-1][:] # for reversed sequence k-1 -> 1
    input_f = Pf[1:][:] # for reversed sequence k -> 2
    
    # sequences: Fs, forward probability(k-1), forward probability(k-1) 
    # results: Bmatrix // initial value = zeros
    [JmFs, Pds, Deltak_l, Deltak_h, Deltak_sig2, Deltak_B], _ = theano.scan(fn = Bmatrix,
                    outputs_info = [None, Pd, None, None, None, None],
                    sequences = [input_Fs, input_f_m1, input_f, dFdsig2, dFdB, dFdl, dFdh, net_input],
                    go_backwards = True                   
                   )
    
    # last timestep of backwards distribution is Pd
    Pb = T.concatenate([Pds[::-1][:], [Pd]],axis=0)
    
    Deltak_l = Deltak_l[::-1]
    Deltak_h = Deltak_h[::-1]
    Deltak_sig2 = Deltak_sig2[::-1]
    Deltak_B = Deltak_B[::-1]

    return JmFs, Pb, Deltak_l, Deltak_h, Deltak_sig2, Deltak_B

## LL = logLike(params, RightClickTimes, LeftClickTimes, Time_bins, rat_choice)

params = [sigma_a, sigma_s, sigma_i, lambda, B, bias, phi, tau_phi, lapse]

In [15]:
# =========== Declare Theano symbolic variables =========== #
## inputs
RightClickTimes = T.dvector("RightClickTimes") # Right Clicks
LeftClickTimes = T.dvector("LeftClickTimes") # Left Clicks
Time_bins = T.vector("Time_bins") # Time_bins
rat_choice = T.wscalar("rat_choice") # rat_choice

In [ ]:
RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, 0)
RClickTimes1

## Testing2

In [16]:
# RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, 0)
# np_Nstep = int(np.ceil(maxT1/dt))
# np_empty_tmp = np.zeros((np_Nstep), dtype=theano.config.floatX)


# theano.config.compute_test_value = 'warn'

# RightClickTimes.tag.test_value = RClickTimes1
# LeftClickTimes.tag.test_value = LClickTimes1
# Time_bins.tag.test_value = np_empty_tmp
# rat_choice.tag.test_value = rat_choice1

theano.config.exception_verbosity = 'high'
theano.config.allow_gc = False
theano.config.optimizer = 'None'
theano.config.NanGuardMode.action='warn'

def logLike_w_manualdiff(params, RightClickTimes, LeftClickTimes, Time_bins, rat_choice):
    """
    :params : arrays of parameters
    : sigma2_a
    : sigma2_s
    : sigma2_i
    : lam 
    : B
    : bias
    : phi
    : tau_phi
    : lapse
    """

    sigma2_a = params[0]
    sigma2_s = params[1]
    sigma2_i = params[2]
    lam = params[3]
    B = params[4]
    bias = params[5]
    phi = params[6]
    tau_phi = params[7]
    lapse = params[8]
            
    # ==== inter-click-intervals and Make adapted clicks
    # there's appreciable same-side adaptation
        
    # input : LeftClickTimes, RightClickTimes
    # output : net_input, total_input
    def make_click_inputs(LeftClickTimes, RightClickTimes, alpha, rho):
        init_val = T.zeros(1)
        
        ici_l = T.extra_ops.diff(LeftClickTimes)
        ici_r = T.extra_ops.diff(RightClickTimes)

        # sequences: T.extra_ops.diff(LeftClickTimes) // inter-click-intervals
        # prior results: lsame (0) // initial value
        # non-sequences: phi, tau_phi // parameters
        Lsame, updates = theano.scan(lambda ici, res, alpha, rho:
                                     1-T.exp((rho*T.log(1-res*alpha)-ici)/rho),
                                      outputs_info = init_val,
                                      sequences=[ici_l],
                                      non_sequences =[alpha, rho],
                                      name = 'Lsame_scan'
                                      )
        Rsame, updates = theano.scan(lambda ici, res, alpha, rho:
                                     1-T.exp((rho*T.log(1-res*alpha)-ici)/rho),
                                      outputs_info = init_val,
                                      sequences=[ici_r],
                                      non_sequences =[alpha, rho],
                                      name = 'Rsame_scan'
                                      )

        Lsame = T.concatenate([T.zeros(1, dtype=Lsame.dtype), Lsame[:,0]],axis=0)
        Rsame = T.concatenate([T.zeros(1, dtype=Rsame.dtype), Rsame[:,0]],axis=0)

        L_here =  T.cast(T.floor((LeftClickTimes+epsilon)/dt),'int32')
        R_here =  T.cast(T.floor((RightClickTimes+epsilon)/dt),'int32')

        ## ==== Collect the adapted click values
        # index starts from 0
        # net_input / total_input

        # sequences: T.floor((LeftClickTimes+epsilon)/dt) // location
        # prior results: zeros // initial value
        # non-sequences: Lsame // parameters
        net_input_l, updates = theano.scan(lambda lhere,lsame, tmp :
                                         T.inc_subtensor(tmp[lhere],lsame),
                                      outputs_info = T.zeros_like(Time_bins),
                                      sequences = [L_here, Lsame],
                                      name = 'net_input_1_scan'
                                      )
        net_input_r, updates = theano.scan(lambda rhere,rsame, tmp :
                                         T.inc_subtensor(tmp[rhere],rsame),
                                      outputs_info = T.zeros_like(Time_bins),
                                      sequences = [R_here, Rsame],
                                      name = 'net_input_r_scan'
                                      )

        net_input = net_input_r[-1] - net_input_l[-1]
        total_input = net_input_r[-1] + net_input_l[-1]
    
        return net_input, total_input

    [net_input, total_input] = make_click_inputs(LeftClickTimes, RightClickTimes, phi, tau_phi)
    
    [net_input_dphi, total_input_dphi] = make_click_inputs(LeftClickTimes, RightClickTimes, phi+ndelta, tau_phi)
    net_input_dphi = (net_input_dphi-net_input)/ndelta
    total_input_dphi = (total_input_dphi-total_input)/ndelta

    [net_input_dtauphi, total_input_dtauphi] = make_click_inputs(LeftClickTimes, RightClickTimes, phi, tau_phi+ndelta)
    net_input_dtauphi = (net_input_dtauphi-net_input)/ndelta
    total_input_dtauphi = (total_input_dtauphi-total_input)/ndelta
    
    ## ==== make bins
    binN = T.cast(T.ceil(B/dx),'int32')
    bin_centers = make_bins(B,dx,binN)
        
    ## ==== make P init
    a0 = T.zeros((bin_centers.shape))
    a0 = T.set_subtensor(a0[binN],1-lapse)
    a0 = T.set_subtensor(a0[0],lapse/2)
    a0 = T.set_subtensor(a0[-1],lapse/2)
    
    ## ==== Fi
    [Fi, dFdsig2_i, dFdB_i, dFdl_i, dFdh_i] = Fmatrix_i([sigma2_i, 0, 0.0], bin_centers)
    
    a = T.dot(Fi,a0)
    a = T.set_subtensor(a[(T.isnan(a)).nonzero()],0)
    
    ai = a
    
    [F0, dFdsig2_0, dFdB_0, dFdl_0, dFdh_0] = Fmatrix([sigma2_a*dt, lam, 0.0], bin_centers)
    
    
    def a_tracing(tot_input, net_input, a, F0, bin_centers, dFdsig2_0, dFdB_0):#, sigma2_a, sigma2_s, lam):
        total_var = sigma2_a*dt + (sigma2_s*tot_input)/40
        [F, dFdsig2, dFdB, dFdl, dFdh] = Fmatrix([total_var, lam, net_input/dt], bin_centers)
        a = T.set_subtensor(a[(T.isnan(a)).nonzero()],0)
        
        a_rest = T.dot(F,a)

        a = T.switch(T.eq(tot_input,0),
                     T.dot(F0,a),
                     a_rest
                     )
        F_res = T.switch(T.eq(tot_input,0),
                     F0,
                     F
                     )
        dFdsig2 = T.switch(T.eq(tot_input,0),
                     dFdsig2_0,
                     dFdsig2
                     )        
        dFdB = T.switch(T.eq(tot_input,0),
                     dFdB_0,
                     dFdB
                     )        
        dFdl = T.switch(T.eq(tot_input,0),
                     dFdl_0,
                     dFdl
                     )             
        dFdh = T.switch(T.eq(tot_input,0),
                     dFdh_0,
                     dFdh
                     )     
        
        a = T.set_subtensor(a[(T.isnan(a)).nonzero()],0)
        
        return a, F_res, dFdsig2, dFdB, dFdl, dFdh
        
    total_input = T.set_subtensor(total_input[(T.isnan(total_input)).nonzero()],0)
    net_input = T.set_subtensor(net_input[(T.isnan(net_input)).nonzero()],0)
    
    # sequences: T.floor((LeftClickTimes+epsilon)/dt) // location
    # prior results: zeros // initial value
    # non-sequences: Lsame // parameters
    [a_res, F_res, dFdsig2, dFdB, dFdl, dFdh], _ = theano.scan(fn = a_tracing,
                             outputs_info = [a, None,None,None,None,None],
                             sequences = [total_input[:-1], net_input[:-1]],
                             non_sequences = [F0,bin_centers, dFdsig2_0, dFdB_0],#,sigma2_a,sigma2_s,lam]
                             name = 'a_tracing_Pf_scan'
                        )
    a = a_res[-1]
    Pf = T.concatenate([[ai],a_res],axis=0)
    
    
    def posterior_probability(a,bin_centers,bias,rat_choice):
    
        bias_hp = T.cast(T.ceil((bias-bin_centers[1])/dx) + 1,'int32') # top
        bias_lp = T.cast(T.floor((bias-bin_centers[1])/dx) + 1,'int32')

        dh = bin_centers[bias_hp] - bias
        dl = bias - bin_centers[bias_lp]
        dd = dh + dl

        # Given probability distribution Pf 
        # Calculate the posterior probability 
        # Pd_r : right side posterior
        # Pd_l : left side posterior

        Pd_r = T.zeros((bin_centers.shape))
        Pd_r = T.set_subtensor(Pd_r[-bias_hp+1:],a[-bias_hp+1:])

        Pd_rest_r = T.set_subtensor(Pd_r[bias_hp],a[bias_hp]*(0.5+dh/dd/2))
        Pd_rest_r = T.set_subtensor(Pd_rest_r[bias_lp],a[bias_lp]*(dh/dd/2))

        Pd_r = T.switch(T.eq(bias_hp,bias_lp),
                      T.set_subtensor(Pd_r[bias_lp],a[bias_lp]/2),
                     Pd_rest_r)

        Pd_l = T.zeros((bin_centers.shape))
        Pd_l = T.set_subtensor(Pd_l[0:bias_lp],a[0:bias_lp])

        Pd_rest_l = T.set_subtensor(Pd_l[bias_hp],a[bias_hp]*(dl/dd/2))
        Pd_rest_l = T.set_subtensor(Pd_rest_l[bias_lp],a[bias_lp]*(0.5+dl/dd/2))

        Pd_l = T.switch(T.eq(bias_hp,bias_lp),
                      T.set_subtensor(Pd_l[bias_lp],a[bias_lp]/2),
                     Pd_rest_l)

        ## rat_choice > 0 : R
        ## rat_choice < 0 : L

        Pd = T.switch(T.gt(rat_choice,0),
                        Pd_r,
                        Pd_l)
    
        return Pd
    
    Pd = posterior_probability(a,bin_centers,bias,rat_choice)
    likely = T.sum(Pd)
    
    JmFs, Pb, Deltak_l, Deltak_h, Deltak_sig2, Deltak_B = backwards(Pf, Pd, F_res, dFdsig2, dFdB, dFdl, dFdh, net_input[:-1])
    
    # consider initial state
    JmF_i = T.ones((bin_centers.shape[0],bin_centers.shape[0]))*a0 / T.transpose(T.ones((bin_centers.shape[0],bin_centers.shape[0]))*Pf[0][:]) * T.transpose(T.ones((bin_centers.shape[0],bin_centers.shape[0]))*Pb[0][:])

    JmF_i = T.set_subtensor(JmF_i[T.or_(T.isnan(JmF_i),T.isinf(JmF_i)).nonzero()],0)
    JmF_i = T.set_subtensor(JmF_i[T.eq(Fi,0).nonzero()],0)
    
    grad_sigma2_i = T.sum(T.sum(JmF_i*dFdsig2_i))
    gard_B = T.sum(T.sum(JmF_i*dFdB_i))+T.sum(Deltak_B)
    
    grad_lam = T.sum(Deltak_l)
    grad_sigma2_a = T.sum(Deltak_sig2)*dt
    grad_sigma2_s = T.sum(Deltak_sig2*total_input[:-1])/total_rate
    
    grad_phi = T.sum(Deltak_h*net_input_dphi[:-1])/dt + T.sum(Deltak_sig2*total_input_dphi[:-1])*sigma2_s/total_rate
    grad_tauphi = T.sum(Deltak_h*net_input_dtauphi[:-1])/dt + T.sum(Deltak_sig2*total_input_dtauphi[:-1])*sigma2_s/total_rate
        
    LLepsilon = posterior_probability(a,bin_centers,bias+ndelta,rat_choice)
    
    grad_bias = (T.log(T.sum(LLepsilon)) - T.log(likely))/ndelta
    grad_lapse = (T.log((likely-lapse/2)/(1-lapse)*(1-(lapse+ndelta))+(lapse+ndelta)/2) - T.log(likely))/ndelta
    
    return T.log(likely), T.stack([grad_sigma2_a, grad_sigma2_s, grad_sigma2_i, 
                                   grad_lam, gard_B, grad_bias, grad_phi, grad_tauphi, grad_lapse])

#     return T.log(pright), T.stack([grad_sigma2_a, grad_sigma2_s,
#                                    grad_sigma2_i, grad_lam, gard_B])



#     return T.log(pright), T.stack([grad_sigma2_a, grad_sigma2_s, grad_sigma2_i, 
#                                    grad_lam, gard_B, gard_bias, grad_phi, grad_tauphi, grad_lapse])

LL, out_grads = logLike_w_manualdiff(params, RightClickTimes, LeftClickTimes, Time_bins, rat_choice)

# all_grads = theano.grad(cost, all_weights, known_grads = {W : masked_grads})
grads = T.grad(LL,params, known_grads = {sigma2_a : out_grads[0],
                                         sigma2_s : out_grads[1],
                                         sigma2_i : out_grads[2],
                                         lam : out_grads[3],
                                         B : out_grads[4],
                                         phi : out_grads[5],
                                         tau_phi : out_grads[6],
                                        })
#[sigma2_a, sigma2_s, sigma2_i, lam, B, bias, lapse]
# out_grads = T.grad(LL,lam)

# grads = nan_to_inf(grads) 
# grads = theano.grad(LL, params, known_grads={z: grads}) 

out_grads = T.stack(*grads)

practice_train = theano.function(
    inputs = [RightClickTimes, LeftClickTimes, Time_bins, rat_choice],
    outputs = [LL, out_grads],
    on_unused_input = 'ignore'
#     ,mode = theano.compile.MonitorMode(pre_func = inspect_inputs, post_func=detect_nan)
#     ,mode = theano.compile.MonitorMode(pre_func = inspect_inputs, post_func=detect_nan)
#     .excluding('local_elemwise_fusion', 'inplace')
#     ,mode = NanGuardMode(nan_is_error=True, inf_is_error=True, big_is_error=True)
)

i = 0
RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, i)
np_Nstep = int(np.ceil(maxT1/dt))
np_empty_tmp = np.zeros((np_Nstep), dtype=None)

# gradi = practice_train(RClickTimes1, LClickTimes1, np_empty_tmp, rat_choice1)
[LLi, gradi] = practice_train(RClickTimes1, LClickTimes1, np_empty_tmp, rat_choice1)
# LLi = practice_train(RClickTimes1, LClickTimes1, np_empty_tmp, rat_choice1)

# print "grad : " + str(gradi)
print str(i+1)+ " - LL : " + str(LLi) + ", grads : " + str(gradi) 


1 - LL : -2.53525394044, grads : [  3.43742048e-01   8.61102340e-02   1.00579392e+00   1.40499903e-01
  -2.65807803e-13   9.82227353e-01   9.82179117e-01  -1.29679116e+00
   5.89979726e+00]


In [17]:
# Training cycle for 27 trials 
# **** make same values as the matlab code.
for i in range(27):
    RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, i)
    np_Nstep = int(np.ceil(maxT1/dt))
    np_empty_tmp = np.zeros((np_Nstep), dtype=None)

    [LLi, gradi] = practice_train(RClickTimes1, LClickTimes1, np_empty_tmp, rat_choice1)
 
#     print str(i+1)+ " - LL : " + str(LLi) + ", grads : " + str(gradi) 
    print [ "{:0.5f}".format(x) for x in gradi ]


print "Done."

['0.34374', '0.08611', '1.00579', '0.14050', '-0.00000', '0.98223', '0.98218', '-1.29679', '5.89980']
['-0.06132', '-0.02141', '-0.18079', '0.02817', '-0.00000', '-0.35468', '-0.35469', '0.63216', '-0.40481']
['-0.00344', '-0.00080', '-0.01314', '-0.00051', '0.00000', '-0.00750', '-0.00750', '0.01853', '-0.52525']
['0.16448', '0.03588', '0.90909', '0.02751', '-0.00000', '-0.48947', '-0.48948', '-0.73245', '8.35771']
['-0.04322', '-0.01462', '-0.27044', '0.00066', '0.00000', '0.32219', '0.32218', '0.35710', '-0.44135']
['0.12023', '0.03510', '0.46424', '0.02075', '-0.00000', '-1.23899', '-1.23907', '-1.52379', '0.73436']
['-0.00002', '-0.00000', '-0.00004', '-0.00000', '0.00000', '0.00002', '0.00002', '0.00007', '-0.52631']
['-0.00014', '-0.00003', '-0.00030', '-0.00002', '0.00000', '-0.00015', '-0.00015', '0.00044', '-0.52630']
['0.15674', '0.04506', '0.98550', '0.09029', '-0.00000', '1.63600', '1.63586', '-0.69130', '1.35303']
['-0.02326', '-0.00757', '-0.05810', '-0.00682', '0.00000'

In [ ]:
LL, out_grads = logLike_w_manualdiff(params, RightClickTimes, LeftClickTimes, Time_bins, rat_choice)

# feed the value to Know Grad
grads = T.grad(LL,params, known_grads = {sigma2_a : out_grads[0],
                                         sigma2_s : out_grads[1],
                                         sigma2_i : out_grads[2],
                                         lam : out_grads[3],
                                         B : out_grads[4],
                                         phi : out_grads[6],
                                         tau_phi : out_grads[7]
                                        })
out_grads = T.stack(*grads)

practice_train = theano.function(
    inputs = [RightClickTimes, LeftClickTimes, Time_bins, rat_choice],
    outputs = [LL, out_grads],
    mode = DebugMode(),
    on_unused_input = 'ignore'
)


i = 0
RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, i)
np_Nstep = int(np.ceil(maxT1/dt))
np_empty_tmp = np.zeros((np_Nstep), dtype=None)

[LLi, gradi] = practice_train(RClickTimes1, LClickTimes1, np_empty_tmp, rat_choice1)

print str(i+1)+ " - LL : " + str(LLi) + ", grads : " + str(gradi) 



# # Training cycle
# for i in range(27):
#     RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, i)
#     np_Nstep = int(np.ceil(maxT1/dt))
#     np_empty_tmp = np.zeros((np_Nstep), dtype=None)

#     [LLi, gradi] = practice_train(RClickTimes1, LClickTimes1, np_empty_tmp, rat_choice1)
 
# #     print str(i+1)+ " - LL : " + str(LLi) + ", grads : " + str(gradi) 
#     print [ "{:0.5f}".format(x) for x in gradi ]


# print "Done."




In [ ]:
LL,gradss = logLike_w_manualdiff(params, RightClickTimes, LeftClickTimes, Time_bins, rat_choice)

# grads = T.grad(LL,params)
grads = T.grad(LL,[sigma2_a, sigma2_s, sigma2_i, lam, B, bias, lapse])
out_grads = T.stack(*grads)

practice_train = theano.function(
    inputs = [RightClickTimes, LeftClickTimes, Time_bins, rat_choice],
    outputs = [LL, out_grads],
    on_unused_input = 'ignore'
)

i = 0
RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, i)
np_Nstep = int(np.ceil(maxT1/dt))
np_empty_tmp = np.zeros((np_Nstep), dtype=None)

[LLi, gradi] = practice_train(RClickTimes1, LClickTimes1, np_empty_tmp, rat_choice1)

print str(i+1)+ " - LL : " + str(LLi) + ", grads : " + str(gradi) 



In [ ]:
theano.config.allow_gc = False
theano.config.optimizer = 'None'
theano.config.NanGuardMode.action='warn'



In [ ]:
np.shape(gradi)

In [ ]:
lapse

In [ ]:
print [ "{:0.5f}".format(x) for x in gradi ]

In [ ]:
gradi[1]

In [ ]:
gradi[:,0]

In [ ]:
theano.config.exception_verbosity

In [ ]:
np_Nstep

In [ ]:
np.arange(10)[1:-1]

In [ ]:
sum(sum(np.ones((10,10))*np.ones((10,10))))